In [1]:
!pip install ollama


In [3]:
# example call

import ollama

OLLAMA_MODEL = "qwen2.5-coder:7b"
response = ollama.chat(
    model=OLLAMA_MODEL,  # or your chosen local model
    messages=[{"role": "user", "content": "Write Python code that adds two numbers"}]
)

print(response['message']['content'])


Sure! Here's a simple Python function that takes two numbers as input and returns their sum:

```python
def add_two_numbers(num1, num2):
    return num1 + num2

# Example usage:
result = add_two_numbers(5, 3)
print("The sum of 5 and 3 is:", result)
```

You can call this function with any two numbers to get their sum.


In [13]:
# Building a Local CodeAgent Skeleton
import ollama
import traceback
import io, contextlib # for execute_code

OLLAMA_MODEL = "qwen2.5-coder:7b"

class LocalOllamaModel:
    def __init__(self, model_name=OLLAMA_MODEL):
        self.model_name = model_name

    def generate(self, messages):
        """Send conversation context to Ollama and return LLM completion."""
        response = ollama.chat(model=self.model_name, messages=messages)
        return response["message"]["content"]


class CodeAgent:
    def __init__(self, tools=None, model=None, authorized_imports=None):
        self.tools = tools or {}
        self.model = model
        self.memory = []
        self.authorized_imports = authorized_imports or []

    def run(self, task: str, max_steps=5):
        # Initialize conversation
        self.memory.append({"role": "system", "content": "You are a code-executing assistant. Output Python code only when needed."})
        self.memory.append({"role": "user", "content": task})

        for step in range(max_steps):
            # 1. Ask LLM for next action
            completion = self.model.generate(self.memory)
            print(f"\n--- Step {step+1} LLM Output ---\n{completion}")

            # 2. Try to extract Python code
            code = self.extract_code(completion)
            if code:
                try:
                    result = self.execute_code(code)
                except Exception as e:
                    result = f"Execution error: {e}\n{traceback.format_exc()}"

                print(f"--- Step {step+1} Execution Result ---\n{result}")

                # Log result back to memory
                self.memory.append({"role": "assistant", "content": completion})
                self.memory.append({"role": "system", "content": f"Execution result: {result}"})
            else:
                # If no code block, assume final answer
                return completion

        return "Max steps reached."

    def extract_code(self, text: str):
        """Naive code block extraction from markdown triple backticks."""
        if "```" in text:
            parts = text.split("```")
            return parts[1] if len(parts) > 1 else None
        return None
  

    def execute_code_hasissues(self, code: str):
        local_env = {}
        local_env.update(self.tools)
        safe_globals = {"__builtins__": __builtins__}
        for imp in self.authorized_imports:
            safe_globals[imp] = __import__(imp)

        buffer = io.StringIO()
        with contextlib.redirect_stdout(buffer):
            exec(code, safe_globals, local_env)
        output = buffer.getvalue()
        return output or local_env.get("result", "✅ Code executed with no output.")

    def execute_code(self, text: str):
        """Extract Python code from markdown triple backticks."""
        if "```" in text:
            parts = text.split("```")
            if len(parts) > 1:
                code = parts[1].strip()
                # 🔑 Remove optional "python" or "py" language tags
                if code.lower().startswith("python"):
                    code = code[6:].strip()
                elif code.lower().startswith("py"):
                    code = code[2:].strip()
                return code
        return None

In [14]:
# Defining Tools Locally, Instead of Hugging Face tools, just pass Python functions:
def suggest_menu(occasion: str) -> str:
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    return "Custom menu for the butler."

agent = CodeAgent(
    tools={"suggest_menu": suggest_menu},
    model=LocalOllamaModel(OLLAMA_MODEL),
    authorized_imports=["datetime"]
)

agent.run("Prepare a formal menu for the party.", max_steps=3)



--- Step 1 LLM Output ---
Certainly! Below is an example of a formal menu that could be prepared for a party. This is just one possible menu; you can modify it based on your preferences, dietary requirements, and the time of day.

---

**[Event Name] Party Menu**

*Appetizers:*
- Bruschetta with Fresh Mozzarella, Cherry Tomatoes, and Basil
- Mini Caprese Salad with Balsamic Glaze
- Shrimp Cocktail with Avocado and Cilantro
- Assorted Deviled Eggs

*First Course:*
- Grilled Salmon Fillets with Lemon-Dill Butter Sauce
- Herb-Crusted Rack of Lamb
- Pan-Seared Duck Breast with Honey Glaze

*Salad:*
- Mixed Greens Salad with Roasted Vegetables, Balsamic Vinaigrette

*Second Course:*
- Beef Wellington with Truffle Sauce and Mashed Potatoes
- Grilled Portobello Mushrooms with Garlic Butter
- Shrimp and Gnocchi in Creamy Alfredo Sauce

*Desserts:*
- Tiramisu with Espresso Liqueur
- Chocolate Fondue with Berries, Marshmallows, and Almonds
- Mini Pecan Pie Bars

*Drinks:*
- Sparkling Water with

'Certainly! Below is an example of a formal menu that could be prepared for a party. This is just one possible menu; you can modify it based on your preferences, dietary requirements, and the time of day.\n\n---\n\n**[Event Name] Party Menu**\n\n*Appetizers:*\n- Bruschetta with Fresh Mozzarella, Cherry Tomatoes, and Basil\n- Mini Caprese Salad with Balsamic Glaze\n- Shrimp Cocktail with Avocado and Cilantro\n- Assorted Deviled Eggs\n\n*First Course:*\n- Grilled Salmon Fillets with Lemon-Dill Butter Sauce\n- Herb-Crusted Rack of Lamb\n- Pan-Seared Duck Breast with Honey Glaze\n\n*Salad:*\n- Mixed Greens Salad with Roasted Vegetables, Balsamic Vinaigrette\n\n*Second Course:*\n- Beef Wellington with Truffle Sauce and Mashed Potatoes\n- Grilled Portobello Mushrooms with Garlic Butter\n- Shrimp and Gnocchi in Creamy Alfredo Sauce\n\n*Desserts:*\n- Tiramisu with Espresso Liqueur\n- Chocolate Fondue with Berries, Marshmallows, and Almonds\n- Mini Pecan Pie Bars\n\n*Drinks:*\n- Sparkling Water

In [16]:
# duckduckgo now used so far, below is just using the package
from ddgs import DDGS

with DDGS() as ddgs:
    results = ddgs.text("party menu ideas", max_results=5)
    for r in results:
        print(r["title"], r["href"])

Party Supplies Online & In-Store | 26 Party City Locations | Party … https://www.partycity.com/?msockid=05e87277e24e6cc7031c641ce3b46d77
Party City Stores – Find Party Supplies & Decorations Near You https://stores.partycity.com/us/
Decorations - Party City https://www.partycity.com/decorations?msockid=05e87277e24e6cc7031c641ce3b46d77
Party Supplies - Party City https://www.partycity.com/party-supplies?msockid=05e87277e24e6cc7031c641ce3b46d77
Theme Parties - Party Supplies & Ideas | Party City https://www.partycity.com/party-themes?msockid=05e87277e24e6cc7031c641ce3b46d77
